In [1]:
import torch as tc
import numpy as np
import pandas as pd

from star.star import STAR
from curve_utils import CurveUtils
from curve_generator import CurveGenerator, get_all_curves, select_better, calculate_derived
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

In [2]:
their_semantics = [
    'Neck girth',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Upper arm girth R',
    'Thigh girth R',
    'Height (m)',
    'Total crotch length',
    'Across back shoulder width',
]
our_semantic = [
    'neck_girth', # 5.3.2
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'upper_arm_girth', # 5.3.16
    'thigh_girth', # 5.3.20
    'stature', # 5.1.1
    'crotch_length', # 5.1.15
    'shoulder_width', # 5.4.3
]
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
    # 'stature': 0,
    # 'crotch_length': 5
}
curves_names = list(curve_index.keys())

# Prepating Bodies

In [3]:
for gender in ['female', 'male']:
    betas = tc.FloatTensor(np.load(f'data/{gender}_betas.npy')).to(device)
    trans = tc.FloatTensor(np.load(f'data/{gender}_trans.npy')).to(device)
    star = STAR(gender=gender, passed_path=f"star/{gender}.npz", num_betas=betas.shape[-1])
    
    # poses = tc.FloatTensor(np.load(f'data/{gender}_poses.npy')).to(device)
    # bodies = star(poses, betas, trans)
    # tc.save(bodies, f'data/{gender}_bodies_a.pt')
    
    poses = np.zeros((betas.shape[0],72))
    poses[:,5] = 6 * np.pi/180
    poses[:,8] = -6 * np.pi/180
    poses = tc.cuda.FloatTensor(poses)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_t.pt')

    poses = np.zeros((betas.shape[0],72))
    poses[:,5] = 30 * np.pi/180
    poses[:,8] = -30 * np.pi/180
    poses = tc.cuda.FloatTensor(poses)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_a.pt')

template = star.faces.to(device)

In [4]:
%%script false --no-raise-error
for gender in ['female', 'male']:
    bodies = tc.load(f'data/{gender}_bodies_t.pt')
    for idx, body in enumerate(bodies):
            CurveUtils.save_obj(f"tests/{gender}_{idx}_body.obj", body, star.faces+1)

In [5]:
selected_subjects = dict()
selected_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.zip')
    measures.index = measures['Subject']
    measures = measures[measures['Measuring station'] == "MOVE4D"]
    temp_measures = measures[their_semantics].iloc[::2]
    
    mean_measures = measures._get_numeric_data().mean(axis=0)
    # selected_subjects[gender] = abs(temp_measures - mean_measures).sum(axis=1).idxmin()
    selected_subjects[gender] = 'IEEEP2_07' if gender == 'female' else 'IEEEP2_04'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])*2
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['stature'] *= 1000

In [6]:
# %%script false --no-raise-error
all_data = []
all_best_curves = []
for gender in ['female', 'male']:
    print(f'MEASURING {gender.upper()} BODY')
    bodies = tc.load(f'data/{gender}_bodies_t.pt')
    body = bodies[selected_subjects[gender]]
    measures = selected_measures[gender]
    result = CurveUtils.get_curves(body, selected_measures['female'], template, device)
    best_curves, data = select_better(result, body, template, measures, device)
    print(data)
    tc.save(best_curves, f'data/{gender}_best_curves.zip')

MEASURING FEMALE BODY
     original    measured  error(mm)
0   35.705200   35.694534   0.106659
1  107.182999  107.329384   1.463852
2   99.165901   99.164696   0.012054
3  115.617004  115.025162   5.918427
4   35.556900   35.690845   1.339455
5   65.943703   65.977676   0.339737
6  162.000000  162.018723   0.187225
7   77.539696   78.377678   8.379822
MEASURING MALE BODY
     original    measured   error(mm)
0   40.458599   44.334217   38.756180
1  110.261002   99.862251  103.987503
2   96.418800   96.426033    0.072327
3  108.165001  108.319160    1.541595
4   33.042099   33.031906    0.101929
5   64.755295   64.736732    0.185623
6  179.000000  180.780807   17.808075
7   78.484100   59.315048  191.690521


In [7]:
all_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.zip')
    all_measures[gender] = dict()
    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        temp_measures = measures[measures['Measuring station'] == station]
        temp_measures = temp_measures[temp_measures['Phase'] == 2]
        temp_measures = temp_measures[their_semantics]
        temp_measures.columns = our_semantic
        temp_measures['stature'] *= 1000
        all_measures[gender][station] = temp_measures

In [11]:
tc.set_printoptions(precision=2, sci_mode=False, linewidth=100)

for gender in ['female', 'male']:
    bodies = tc.load(f'data/{gender}_bodies_t.pt')
    bodies_a = tc.load(f'data/{gender}_bodies_a.pt')
    best_curves = tc.load(f'data/female_best_curves.zip')

    all_bodies_measures = []
    for idx,body in enumerate(bodies):
        all_curves_measures = []
        all_positions = []
        for curve in best_curves:
            position = CurveGenerator.generate_positions(body, curve)
            position, measures, temp_pos = CurveUtils.ray_polygon_cosilion(body[template], position)
            all_positions.append(position)
            measures = CurveGenerator.calculate_distances(position)
            all_curves_measures.append(measures)
        
        # derived_measures, derived_positions = calculate_derived(body, template, all_positions, device)
        # all_curves_measures.extend(derived_measures)
        # all_positions.extend(derived_positions)
        # all_positions = derived_positions
        all_bodies_measures.append(all_curves_measures)
        CurveUtils.save_obj(f"tests/{gender}_{idx}_curves.obj", tc.row_stack(all_positions))
    all_bodies_measures = tc.FloatTensor(all_bodies_measures)
    our_measures = all_bodies_measures

    tempdata = pd.read_pickle(f'data/{gender}_measures.zip')
    tempdata = tempdata[tempdata['Measuring station'] == 'MOVE4D']
    tempdata = tempdata['Age (years)']
    data_to_save = pd.DataFrame(our_measures, index=tempdata.index,columns=curves_names)
    data_to_save['age'] = tempdata
    print(data_to_save)
    data_to_save.to_csv(f'data/{gender}_calculated_measures.csv')


    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        print(f"\n\nmeasures difference from {station}:")
        
        ieee_measures = tc.FloatTensor(all_measures[gender][station][curves_names].to_numpy())
        diff = abs(ieee_measures - (our_measures*10))

        print(pd.DataFrame({
            "mean": diff.mean(axis=0),
            "std": diff.std(axis=0),
            "min": diff.min(axis=0).values,
            "max": diff.max(axis=0).values,
        }, index=curves_names))

      neck_girth  bust_chest_girth  waist_girth   hip_girth  upper_arm_girth  \
724    31.809977         87.649536    81.709778  105.947807        30.142504   
725    31.563108         88.027229    81.801163  105.923370        30.619509   
759    30.591846         88.843552    73.215401   94.412849        28.542591   
760    30.387405         87.707085    74.207611   94.896683        28.401756   
830    35.680706        105.710495    99.112511  114.847061        35.699589   
...          ...               ...          ...         ...              ...   
1926   35.990231        103.927437    92.375885   97.410492        30.822271   
1943   34.155048         98.346794    89.486305  106.434082        30.989843   
1944   34.412159         98.886803    88.317238  106.218376        30.632364   
1979   40.491886        117.934799   111.876724  132.884140        40.012039   
1980   40.441242        117.426369   111.111259  132.206436        40.111851   

      thigh_girth  age  
724     61.445

In [9]:
'''
DONE neck_girth 
neck_to_waist
DONE upper_arm_girth
neck_to_wrist
shoulder_width
DONE bust_chest_girth
DONE waist_girth
DONE hip_girth
DONE thigh_girth
crotch_length
leg_height
'''

'\nDONE neck_girth \nneck_to_waist\nDONE upper_arm_girth\nneck_to_wrist\nshoulder_width\nDONE bust_chest_girth\nDONE waist_girth\nDONE hip_girth\nDONE thigh_girth\ncrotch_length\nleg_height\n'

In [10]:
for gender in ["female", "male"]:
    star = STAR(gender=gender, passed_path=f"star/{gender}.npz")
    poses = tc.zeros((1,72)).to(device)
    betas = tc.zeros((1,10)).to(device)
    trans = tc.zeros((1,3)).to(device)
    body = star(poses, betas, trans)[0]